In [3]:
# generate large N dataset, compute scaling, compare to smaller N (100, 1000, 10000)
# generate, simulate, and plot in one go so we can leave it overnight 
# test with N = 100,200,300 

%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt 
import experiment as ex
from network_generation import generation as ng
from scipy import optimize
from scipy.odr import * 

In [4]:
# params 

# above connectivity threshold 
kappa_range = [10,15,20]
kappa_range_fine = np.arange(.1,22,.01)
n_range = [100,1000,10000]
d_range = [2,9,25]#[2,3,6,9,15,25]
boundaries = ['p'] 
boundary = boundaries[0]
num_samples = 10
num_r = 3

In [4]:
# generate data 

## RGGs ##
print 'running ' + str(len(n_range)*len(d_range)*len(boundaries)*len(kappa_range)) + ' generations'
i=1
for kappa in kappa_range:
    for n in n_range:
        for d in d_range:
            for boundary in boundaries: 
                RGG = ng.RGGEnsemble(kappa, n, d, boundary=boundary, num_radii=num_r)
                RGG.generate_samples(n=num_samples)
                # data to disk 
                RGG.to_disk()
                print 'i = ' + str(i) + ': n = ' + str(n) + ', d = ' + str(d) + ', kappa = ' + str(kappa) 
                i+=1 
                
###########################################

# generate ER data 
print 'running ' + str(len(n_range)*len(kappa_range)) + ' generations'
i=1
for kappa in kappa_range:
    for n in n_range:
        # make an ER Ensemble
        ER = ng.EREnsemble(kappa,n)

        # generate samples 
        # sample data is stored in the object 
        ER.generate_samples(n=num_samples)

        # data to disk 
        ER.to_disk()
        print 'saved run ' + str(i)
        i+=1 

running 27 generations
i = 1: n = 100, d = 2, kappa = 10
i = 2: n = 100, d = 9, kappa = 10
i = 3: n = 100, d = 25, kappa = 10
i = 4: n = 1000, d = 2, kappa = 10
i = 5: n = 1000, d = 9, kappa = 10
i = 6: n = 1000, d = 25, kappa = 10
i = 7: n = 10000, d = 2, kappa = 10
i = 8: n = 10000, d = 9, kappa = 10
i = 9: n = 10000, d = 25, kappa = 10
i = 10: n = 100, d = 2, kappa = 15
i = 11: n = 100, d = 9, kappa = 15
i = 12: n = 100, d = 25, kappa = 15
i = 13: n = 1000, d = 2, kappa = 15
i = 14: n = 1000, d = 9, kappa = 15
i = 15: n = 1000, d = 25, kappa = 15
i = 16: n = 10000, d = 2, kappa = 15
i = 17: n = 10000, d = 9, kappa = 15
i = 18: n = 10000, d = 25, kappa = 15
i = 19: n = 100, d = 2, kappa = 20
i = 20: n = 100, d = 9, kappa = 20
i = 21: n = 100, d = 25, kappa = 20
i = 22: n = 1000, d = 2, kappa = 20
i = 23: n = 1000, d = 9, kappa = 20
i = 24: n = 1000, d = 25, kappa = 20
i = 25: n = 10000, d = 2, kappa = 20
i = 26: n = 10000, d = 9, kappa = 20
i = 27: n = 10000, d = 25, kappa = 20
runni

In [1]:
# set up plots and axes 
ndkfig = plt.figure(1)
ndkax = ndkfig.add_subplot(1,1,1)
gdfig = plt.figure(2)
gdax = gdfig.add_subplot(1,1,1)

# MODEL
def f(B, x):
    return np.exp(B[0]*x)

gdhandles = []
ndkhandles = []
gdlegend = []
ndklegend = []

# ER
ERE = ex.ERExperiment(kappa_range, n_range[-1])
mean_degree_list, std_degree_list, _ = ERE.find_degree_stats()
mean_nD_list, std_nD_list, _ = ERE.find_nD_stats()

x = mean_degree_list
y = mean_nD_list 
wx = [s+1 for s in std_degree_list]
wy = [s+1 for s in std_nD_list] 

exponential = Model(f)
data = Data(x, y, wd=wx, we=wy)
odr = ODR(data, exponential, beta0=[-0.5])
output = odr.run()

ndkhandle, = ndkax.plot(kappa_range_fine,f(output.beta,kappa_range_fine),'r--')
    
ndkhandles.append(ndkhandle)
ndklegend.append('ER Simulation')

ERline, = gdax.plot([min(d_range),max(d_range)],[output.beta[0],output.beta[0]],'r-')
gdax.plot([min(d_range),max(d_range)],[output.beta[0]+output.sd_beta[0],output.beta[0]+output.sd_beta[0]],'r--')
gdax.plot([min(d_range),max(d_range)],[output.beta[0]-output.sd_beta[0],output.beta[0]-output.sd_beta[0]],'r--')
gdhandles.append(ERline)
gdlegend.append('ER gamma')

print 'ER sim done', 'gamma = ' + str(output.beta[0]) + ' +/- ' + str(output.sd_beta[0])

# RGG
for n in n_range:
    gammas = []
    gamma_errs = []
    for d in d_range:
        RGG = ex.RGGExperiment(kappa_range, n, d, shortcut_prob=0, boundary=boundary, num_radii=num_r)

        mean_degree_list, std_degree_list, stddev_degree_list = RGG.find_degree_stats()
        mean_nD_list, std_nD_list, stddev_nD_list = RGG.find_nD_stats()

        print std_nD_list
        print std_degree_list
        
        x = mean_degree_list
        y = mean_nD_list

        wx = [s+1 for s in std_degree_list]
        wy = [s+1 for s in std_nD_list] 

        exponential = Model(f)
        data = Data(x, y, wd=wx, we=wy)
        odr = ODR(data, exponential, beta0=[-0.5])
        output = odr.run()

        ndkhandle, = ndkax.plot(kappa_range_fine,f(output.beta,kappa_range_fine),'-')
        ndkhandles.append(ndkhandle)
        ndklegend.append('d='+str(d)+', n='+str(n))
        
        gammas.append(output.beta[0])
        # 95% confidence is 2*standard error 
        print 'error' + str(output.sd_beta[0])
        gamma_errs.append(output.sd_beta[0])
        print str(d) +'th dimension done'

    gdhandle = gdax.errorbar(d_range,gammas,yerr=gamma_errs,fmt='o')
    gdhandles.append(gdhandle)
    gdlegend.append('n='+str(n))
    
ndkax.set_xlabel('Average Mean Degree $\\langle{k}\\rangle$')
ndkax.set_ylabel('Fraction of Driver Nodes $n_D$')
ndkax.legend(ndkhandles,ndklegend)

gdax.set_xlabel('Dimension $d$')
gdax.set_ylabel('Scaling Parameter $\\gamma$')
gdax.legend(gdhandles,gdlegend)

ndkfig.savefig('./plots/k_nD_over_N_BC_' + boundary + '.eps',dpi=800)
gdfig.savefig('./plots/gamma_d_over_N_BC_' + boundary + '.eps',dpi=800)

NameError: name 'plt' is not defined